In [1]:
import cv2
from matplotlib import pyplot  as plt
import numpy as np
from math import cos 
# from _future_ import division

In [2]:
image1 = cv2.imread('/home/shanu/Desktop/berry.jpg',1)
image2 = cv2.imread('/home/shanu/Desktop/qr-code-pack.jpg',1)
image3 = cv2.imread('/home/shanu/Desktop/Qr-code-ver-10.png',1)
image4 = cv2.imread('/home/shanu/Desktop/image.jpeg',1)
image5 = cv2.imread('/home/shanu/Desktop/Wifi_joiner_QR_code.png',1)
images = [image1,image2,image3,image4,image5]
image1.shape

(243, 220, 3)

In [3]:
def printImage(img):
    cv2.imshow('image',img)
    cv2.waitKey()
#     cv2.destroyAllWindows()

In [4]:
def printGrayScaled(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
    cv2.imshow('gray',gray)
    cv2.waitKey()

In [5]:
def rescaling1(image):
    max_dimension = max(image.shape)
    scale = 500.0/max_dimension 
    if scale < 1.0: 
        image = cv2.resize(image,None,fx=scale,fy=scale,interpolation=cv2.INTER_AREA )
        print image.shape
#         printGrayScaled(image)
#         cv2.destroyAllWindows()
        return image
    else:
        image = cv2.resize(image,None,fx=scale,fy=scale,interpolation = cv2.INTER_CUBIC)
        print image.shape
#         printGrayScaled(image)
#         cv2.destroyAllWindows()
        return image

In [6]:
rescaledImgs=[]
for image in images:
    rescaledImgs.append(rescaling1(image))
#     printImage(image)
cv2.destroyAllWindows()    

(500, 453, 3)
(500, 500, 3)
(500, 500, 3)
(500, 375, 3)
(500, 300, 3)


In [7]:
def adaptiveThresh(img):
    thresh = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,5,2)
    printImage(thresh)

In [8]:
def otsuThresh(img): 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     printGrayScaled(img)
    blur = cv2.GaussianBlur(gray,(7,5),0)
    printImage(blur)
#     cv2.destroyAllWindows()
    ret2,th2 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)    # convert to grayscale(binary image) 
    printImage(th2)
    return th2
#     print ret2,th2 
#     cv2.destroyAllWindows()

In [9]:
def cannyEdge(image):
    edged = cv2.Canny(image, 50, 50)
    return edged

In [10]:
# for image in rescaledImgs:
#     gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#     th = adaptiveThresh(gray)
#     printImage(th)
image = rescaling1(images[2])
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#hsv image
# hsv_img = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
# printImage(hsv_img)
# cv2.destroyAllWindows()
th = otsuThresh(image)
printImage(th)
edged = cannyEdge(th)
printImage(edged)
cv2.destroyAllWindows()
cont = edged
# cv2.imwrite('edgedImage3.png',edged)
contouredIm,contour,hierarchy = cv2.findContours(cont,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# contour = np.array(contour)
# print contour.shape
hierarchy = hierarchy[0]
# print hierarchy
# contour = sorted(contour, key=cv2.contourArea, reverse = True)[:3] 
# print contour
'''
for component in zip(contour, hierarchy):
    currentContour = component[0]
    currentHierarchy = component[1]
    x,y,w,h = cv2.boundingRect(currentContour)
    if currentHierarchy[3] < 0:            
        cv2.rectangle(edged,(x,y),(x+w,y+h),(0,255,0),4)
'''        
# printImage(edged)         
        
#     elif currentHierarchy[3] < 0:
#         cv2.rectangle(cont,(x,y),(x+w,y+h),(0,255,0),3)
# printImage(final_img)        
# contours = sorted(contours,key = contours.)
# res_img = cv2.drawContours(image, contour, -1, (0,255,0),1)
# printImage(res_img)
'''
contour = sorted(contour, key=cv2.contourArea, reverse = True)[:3]                             # ! since there are three big rectangles with contours quite
rect_count = []                                                                       
for c in contour:                                     
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02*peri, True)

    if len(approx) == 4: 
        rect_count.append(approx) 
rect_count = np.array(rect_count)
print rect_count
final_img = cv2.drawContours(cont, rect_count,-1, (0,255,0), 3)
printImage(final_img)     #   big and hence differentiable than the others. 
for c in contour:                                     
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02*peri, True)

    if len(approx) == 4: 
        rect_count.append(approx) 
rect_count = np.array(rect_count)
print rect_count
final_img = cv2.drawContours(cont, rect_count,-1, (0,255,0), 3)
printImage(final_img)
cv2.destroyAllWindows()
'''

(500, 500, 3)


'\ncontour = sorted(contour, key=cv2.contourArea, reverse = True)[:3]                             # ! since there are three big rectangles with contours quite\nrect_count = []                                                                       \nfor c in contour:                                     \n    peri = cv2.arcLength(c, True)\n    approx = cv2.approxPolyDP(c, 0.02*peri, True)\n\n    if len(approx) == 4: \n        rect_count.append(approx) \nrect_count = np.array(rect_count)\nprint rect_count\nfinal_img = cv2.drawContours(cont, rect_count,-1, (0,255,0), 3)\nprintImage(final_img)     #   big and hence differentiable than the others. \nfor c in contour:                                     \n    peri = cv2.arcLength(c, True)\n    approx = cv2.approxPolyDP(c, 0.02*peri, True)\n\n    if len(approx) == 4: \n        rect_count.append(approx) \nrect_count = np.array(rect_count)\nprint rect_count\nfinal_img = cv2.drawContours(cont, rect_count,-1, (0,255,0), 3)\nprintImage(final_img)\nc

SyntaxError: invalid syntax (<ipython-input-143-88fbb1a0a128>, line 1)

In [14]:
count = 0
cont_list = []
print hierarchy
print "\n----------------------------------------------\n"
for i in range(len(contour)):
    #check if it is rectangle
    print contour[i].shape
    peri = cv2.arcLength(contour[i],True)
    approx = cv2.approxPolyDP(contour[i],peri*0.015,True)
    
    if cv2.contourArea(contour[i]) and len(approx)==4:  
        count = count+1
        print contour[i]
        print "\n"
        print i
        print "\n"
        print hierarchy[i][0],hierarchy[i][1],hierarchy[i][2], hierarchy[i][3] 
        cont_list.append(contour[i])
#         print "\n"
#         print i
#         print hierarchy[i][3]
        print "\n----------------------------------------------\n"  
cont_list= np.array(cont_list)   
image = cv2.drawContours(image, cont_list,-1,(0,255,0),3)
cv2.imwrite('markerImage.png',image)
printImage(image)

# cv2.
# x,y,w,h = cv2.boundingRect(contour[2])
# print cv2.contourArea(contour[2])
# test_img = cv2.rectangle(edged,(x,y),(x+w,y+h),(0,255,0),4) 
# printImage(test_img)
cv2.destroyAllWindows()

[[  6  -1   1  -1]
 [ -1  -1   2   0]
 [ -1  -1   3   1]
 ..., 
 [ -1  -1 420 418]
 [ -1  -1 421 419]
 [ -1  -1  -1 420]]

----------------------------------------------

(12, 1, 2)
[[[ 59 394]]

 [[ 58 395]]

 [[ 57 395]]

 [[ 57 396]]

 [[ 56 397]]

 [[ 56 441]]

 [[ 57 442]]

 [[104 442]]

 [[104 396]]

 [[103 395]]

 [[102 395]]

 [[101 394]]]


0


6 -1 1 -1

----------------------------------------------

(8, 1, 2)
[[[ 59 394]]

 [[101 394]]

 [[104 397]]

 [[104 441]]

 [[103 442]]

 [[ 57 442]]

 [[ 56 441]]

 [[ 56 397]]]


1


-1 -1 2 0

----------------------------------------------

(12, 1, 2)
(12, 1, 2)
(14, 1, 2)
[[[ 72 408]]

 [[ 71 409]]

 [[ 70 409]]

 [[ 70 410]]

 [[ 69 411]]

 [[ 69 426]]

 [[ 70 427]]

 [[ 70 428]]

 [[ 71 429]]

 [[ 89 429]]

 [[ 90 428]]

 [[ 90 409]]

 [[ 89 409]]

 [[ 88 408]]]


4


-1 -1 5 3

----------------------------------------------

(8, 1, 2)
(14, 1, 2)
(8, 1, 2)
(14, 1, 2)
(12, 1, 2)
(46, 1, 2)
(34, 1, 2)
(26, 1, 2)
(20, 1, 2)
(19, 1,

In [ ]:
def SearchPattern(contours , hierarchy , _dict , level):
    if((level==3 and hierarchy[2]!=-1) or (hierarchy[0]!=-1 and hierarchy[0]!=-1) or (level<3 and hierarchy[2]==-1)):
        return 0
    peri = cv2.arcLength(contours,True)
    approx = cv2.approxPolyDP(contours,peri*0.02,True)
    if  cv2.contourArea(contours) and len(approx)==4:
        if level<3 and SearchPattern(contours[hierarchy[2]],hierarchy[hierarchy[2]],_dict, level+1 ):
            _dict{level} = cv2.arcLength(contours,True)
            return 1
    else:
            return 0            

In [ ]:
def findPattern(contours, hierarchy,countFP):
    if hierarchy[1]==-1 and hierarchy[0]==-1:
        findPattern(contours[hierarchy[2]],hierarchy[hierarchy[2]],countFP )
    #check if it is rectangle 
    peri = cv2.arcLength(contour[i],True)
    approx = cv2.approxPolyDP(contour[i],peri*0.02,True)
    if  cv2.contourArea(contour[i]) and len(approx)==4:
        level = 1
        dictionary = {level:cv2.contourArea(contour[i])}
        if hierarchy[3]!=-1 and SearchPattern(contours[hierarchy[2]],hierarchy[hierarchy[2]], dictionary,level+1): 
            #logic for further evaluation for condition of barCode
            if dictionary{1}/dictionary{2} == 7.0/5.0 and dictionary{2}/dictionary{3} == 5.0/3.0:
                 countFP+1   
    else:     
        findPattern(contours[hierarchy[1]],hierarchy[hierarchy[1]],count   
    return count_FP

In [ ]:
def MorphoTransform(img):
    
    

In [ ]:
#define masking so that image contains only black and white region
# printGrayScaled(image[3])
# printImage(image[3])
# image = cv2.resize(image[3],(640,480))
# printImage()
# printGrayScaled(image)